In [1]:
# Importing all necessary libraries for the proper scrubbing of all data
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import time

In [ ]:
# Creating 
data = pd.DataFrame()
for i in range(2002,2020):
    url_text = requests.get(f'https://www.pro-football-reference.com/years/{i}/opp.htm').text
    soup = BeautifulSoup(url_text)
    teams = []
    iterator = 0
    for x in range(0,32):
        teams.append(soup.find_all('td')[iterator].text)
        iterator += 27
    data[f'{i}'] = teams
data.to_csv('Defense.csv')

In [19]:
#This will be a function which will be called which correlates a teams 3 letter acronym to their actual team
def findTeam(name,year):
    team = ''
    rank = 420
    if name == 'ARI': team = 'Arizona Cardinals'
    elif name == 'ATL': team = 'Atlanta Falcons'
    elif name == 'BAL': team = 'Baltimore Ravens'
    elif name == 'BUF': team = 'Buffalo Bills'
    elif name == 'CAR': team = 'Carolina Panthers'
    elif name == 'CHI': team = 'Chicago Bears'
    elif name == 'CIN': team = 'Cincinnati Bengals'
    elif name == 'CLE': team = 'Cleveland Browns'
    elif name == 'DAL': team = 'Dallas Cowboys'
    elif name == 'DEN': team = 'Denver Broncos'
    elif name == 'DET': team = 'Detroit Lions'
    elif name == 'GNB': team = 'Green Bay Packers'
    elif name == 'HOU': team = 'Houston Texans'
    elif name == 'IND': team = 'Indianapolis Colts'
    elif name == 'JAX': team = 'Jacksonville Jaguars'
    elif name == 'KAN': team = 'Kansas City Chiefs'
    elif name == 'MIA': team = 'Miami Dolphins'
    elif name == 'MIN': team = 'Minnesota Vikings'
    elif name == 'NWE': team = 'New England Patriots'
    elif name == 'NOR': team = 'New Orlean Saints'
    elif name == 'NYG': team = 'New York Giants'
    elif name == 'NYJ': team = 'New York Jets'
    elif name == 'LVR': team = 'Las Vegas Raiders'
    elif name == 'PHI': team = 'Philadelphia Eagles'
    elif name == 'PIT': team = 'Pittsburgh Steelers'
    elif name == 'LAC': team = 'Los Angeles Chargers'
    elif name == 'SFO': team = 'San Francisco 49ers'
    elif name == 'SEA': team = 'Seattle Seahawks'
    elif name == 'LAR': team = 'Los Angeles Rams'
    elif name == 'TAM': team = 'Tampa Bay Buccaneers'
    elif name == 'TEN': team = 'Tennessee Titans'
    elif name == 'WAS': team = 'Washington Football Team'
    #Special Boys
    elif name == 'SDG': team = 'San Diego Chargers'
    elif name == 'STL': team = 'St. Louis Rams'
    elif name == 'OAK': team = 'Oakland Raiders'
    data = pd.read_csv(r'C:\Users\Parker Hill\Desktop\ECE 196\NFL Running Back\Defense.csv')
    frame = pd.DataFrame(data)
    for i in range(0,32):
        iterativeteam = frame[str(year)][i]
        if iterativeteam == team:
            rank = i
            break
    return rank + 1

In [ ]:
#https://www.pro-football-reference.com/years/2002/rushing.htm
#Need to make it so that names and acronyms are saved for in a list so that they can be referenced for urls
#Need to make it so that if the player is not RB or rb, then it skips
rb_data = pd.DataFrame()
rb_data['Year'] = []
rb_data['Name'] = []
rb_data['URLName']= []
for year in range(2002,2020):
    url_text = requests.get(f'https://www.pro-football-reference.com/years/{year}/rushing.htm').text
    soup = BeautifulSoup(url_text)
    iterator = 0
    years = []
    name = []
    url = []
    for x in range(0,32):
        newstr = ''
        urlname = ''
        title = soup.find_all('td')[iterator+3].text
        if title != 'RB' and title != 'rb'and title != 'fb' and title != 'FB':
            iterator += 14
            print(title)
            continue
        player = soup.find_all('td')[iterator].text
        alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz '
        for i in range(0,len(player)):
            if alphabet.find(player[i]) != -1:
                newstr += player[i]
        for i in range(0,len(newstr)):
            if newstr[i] == ' ':
                urlname = newstr[i+1]+newstr[i+2]+newstr[i+3]+newstr[i+4]
                break
        urlname = urlname + player[0] + player[1] + '00'
        new_row = {'Year':year,'Name':newstr,'URLName':urlname}
        rb_data = rb_data.append(new_row,ignore_index = True)
        iterator += 14
print(rb_data)

In [ ]:
#Run to commit running back information into a csv file
rb_data.to_csv('RunningBacks.csv')

In [60]:
newrbdata = pd.read_csv(r'C:\Users\Parker Hill\Desktop\ECE 196\NFL Running Back\RunningBacks.csv')
rbframe = pd.DataFrame(newrbdata)
gamedata = pd.DataFrame()
gamedata['Week'] = []
gamedata['Opponent'] = []
gamedata['Rank'] = []
gamedata['Fantasy']=[]
length = rbframe['Year'].count()
for i in range(0,length):
    year = int(rbframe['Year'][i])
    urlname = rbframe['URLName'][i]
    playername = rbframe['Name'][i]
    url_text = requests.get(f'https://www.pro-football-reference.com/players/W/{urlname}/fantasy/{year}/').text
    soup = BeautifulSoup(url_text)
    titlename = soup.find_all('span')[7].text
    if str.strip(playername)!=str.strip(titlename):
        continue
    for gamenum in range(0,16):
        try:
            week = soup.find_all('td',attrs={"class":"right ","data-stat":"game_num"})[gamenum].text
            opponent = soup.find_all('td',attrs={"class":"left ","data-stat":"opp"})[gamenum].text
            defenserank = findTeam(opponent,year)
            if defenserank == 421:
                continue
            fantasy = soup.find_all('td',attrs={"class":"right ","data-stat":"fantasy_points"})[gamenum].text
            if len(str.strip(fantasy)) == 0:
                fantasy = '0'
            elif float(fantasy) > 70:
                break
            new_row = {'Week':week,'Opponent':opponent,'Rank':defenserank,'Fantasy':fantasy}
            print(new_row)
            gamedata = gamedata.append(new_row,ignore_index = True)
            week = soup.find_all('td',attrs={"class":"right ","data-stat":"game_num"})[gamenum+1].text
        except IndexError:
            break


{'Week': '1', 'Opponent': 'DET', 'Rank': 31, 'Fantasy': '23.10'}
{'Week': '2', 'Opponent': 'IND', 'Rank': 7, 'Fantasy': '25.40'}
{'Week': '3', 'Opponent': 'NYJ', 'Rank': 14, 'Fantasy': '23.40'}
{'Week': '4', 'Opponent': 'KAN', 'Rank': 28, 'Fantasy': '16.60'}
{'Week': '5', 'Opponent': 'NWE', 'Rank': 17, 'Fantasy': '8.70'}
{'Week': '6', 'Opponent': 'DEN', 'Rank': 15, 'Fantasy': '19.90'}
{'Week': '7', 'Opponent': 'BUF', 'Rank': 27, 'Fantasy': '11.90'}
{'Week': '8', 'Opponent': 'GNB', 'Rank': 12, 'Fantasy': '7.00'}
{'Week': '9', 'Opponent': 'NYJ', 'Rank': 14, 'Fantasy': '6.90'}
{'Week': '10', 'Opponent': 'BAL', 'Rank': 19, 'Fantasy': '25.00'}
{'Week': '11', 'Opponent': 'SDG', 'Rank': 22, 'Fantasy': '27.10'}
{'Week': '12', 'Opponent': 'BUF', 'Rank': 27, 'Fantasy': '35.50'}
{'Week': '13', 'Opponent': 'CHI', 'Rank': 25, 'Fantasy': '31.60'}
{'Week': '14', 'Opponent': 'OAK', 'Rank': 6, 'Fantasy': '12.00'}
{'Week': '15', 'Opponent': 'MIN', 'Rank': 30, 'Fantasy': '8.30'}
{'Week': '16', 'Opponent'

{'Week': '6', 'Opponent': 'CAR', 'Rank': 5, 'Fantasy': '6.60'}
{'Week': '7', 'Opponent': 'ARI', 'Rank': 29, 'Fantasy': '8.20'}
{'Week': '8', 'Opponent': 'SEA', 'Rank': 23, 'Fantasy': '15.20'}
{'Week': '9', 'Opponent': 'DET', 'Rank': 31, 'Fantasy': '3.90'}
{'Week': '10', 'Opponent': 'IND', 'Rank': 7, 'Fantasy': '2.20'}
{'Week': '11', 'Opponent': 'JAX', 'Rank': 9, 'Fantasy': '14.50'}
{'Week': '13', 'Opponent': 'SFO', 'Rank': 18, 'Fantasy': '18.60'}
{'Week': '14', 'Opponent': 'NYG', 'Rank': 3, 'Fantasy': '3.00'}
{'Week': '15', 'Opponent': 'PHI', 'Rank': 2, 'Fantasy': '3.60'}
{'Week': '1', 'Opponent': 'PIT', 'Rank': 16, 'Fantasy': '4.00'}
{'Week': '2', 'Opponent': 'NYJ', 'Rank': 14, 'Fantasy': '6.90'}
{'Week': '3', 'Opponent': 'KAN', 'Rank': 28, 'Fantasy': '17.20'}
{'Week': '4', 'Opponent': 'SDG', 'Rank': 22, 'Fantasy': '9.00'}
{'Week': '5', 'Opponent': 'MIA', 'Rank': 4, 'Fantasy': '1.80'}
{'Week': '6', 'Opponent': 'GNB', 'Rank': 12, 'Fantasy': '8.90'}
{'Week': '7', 'Opponent': 'DEN', 'Ran

{'Week': '1', 'Opponent': 'NWE', 'Rank': 17, 'Fantasy': '2.90'}
{'Week': '2', 'Opponent': 'OAK', 'Rank': 6, 'Fantasy': '1.50'}
{'Week': '3', 'Opponent': 'CLE', 'Rank': 10, 'Fantasy': '1.30'}
{'Week': '5', 'Opponent': 'CIN', 'Rank': 32, 'Fantasy': '6.50'}
{'Week': '6', 'Opponent': 'IND', 'Rank': 7, 'Fantasy': '8.70'}
{'Week': '7', 'Opponent': 'BAL', 'Rank': 19, 'Fantasy': '17.30'}
{'Week': '8', 'Opponent': 'CLE', 'Rank': 10, 'Fantasy': '17.30'}
{'Week': '9', 'Opponent': 'ATL', 'Rank': 8, 'Fantasy': '12.00'}
{'Week': '10', 'Opponent': 'TEN', 'Rank': 11, 'Fantasy': '5.30'}
{'Week': '11', 'Opponent': 'CIN', 'Rank': 32, 'Fantasy': '1.70'}
{'Week': '12', 'Opponent': 'JAX', 'Rank': 9, 'Fantasy': '1.40'}
{'Week': '13', 'Opponent': 'HOU', 'Rank': 20, 'Fantasy': '9.60'}
{'Week': '14', 'Opponent': 'CAR', 'Rank': 5, 'Fantasy': '7.10'}
{'Week': '15', 'Opponent': 'TAM', 'Rank': 1, 'Fantasy': '2.10'}
{'Week': '16', 'Opponent': 'BAL', 'Rank': 19, 'Fantasy': '23.70'}
{'Week': '2', 'Opponent': 'SEA', 'R

{'Week': '14', 'Opponent': 'TEN', 'Rank': 13, 'Fantasy': '10.10'}
{'Week': '15', 'Opponent': 'MIA', 'Rank': 3, 'Fantasy': '5.30'}
{'Week': '16', 'Opponent': 'NWE', 'Rank': 1, 'Fantasy': '4.30'}
{'Week': '2', 'Opponent': 'ARI', 'Rank': 32, 'Fantasy': '12.20'}
{'Week': '3', 'Opponent': 'STL', 'Rank': 17, 'Fantasy': '6.60'}
{'Week': '4', 'Opponent': 'GNB', 'Rank': 11, 'Fantasy': '15.50'}
{'Week': '5', 'Opponent': 'SFO', 'Rank': 21, 'Fantasy': '12.30'}
{'Week': '6', 'Opponent': 'CHI', 'Rank': 22, 'Fantasy': '24.20'}
{'Week': '7', 'Opponent': 'CIN', 'Rank': 28, 'Fantasy': '19.80'}
{'Week': '8', 'Opponent': 'PIT', 'Rank': 16, 'Fantasy': '12.70'}
{'Week': '10', 'Opponent': 'DET', 'Rank': 26, 'Fantasy': '19.70'}
{'Week': '11', 'Opponent': 'BAL', 'Rank': 6, 'Fantasy': '7.20'}
{'Week': '12', 'Opponent': 'CLE', 'Rank': 12, 'Fantasy': '19.70'}
{'Week': '13', 'Opponent': 'MIN', 'Rank': 23, 'Fantasy': '10.40'}
{'Week': '14', 'Opponent': 'STL', 'Rank': 17, 'Fantasy': '19.40'}
{'Week': '15', 'Opponent

{'Week': '15', 'Opponent': 'DAL', 'Rank': 2, 'Fantasy': '10.20'}
{'Week': '16', 'Opponent': 'CAR', 'Rank': 10, 'Fantasy': '3.70'}
{'Week': '1', 'Opponent': 'ATL', 'Rank': 30, 'Fantasy': '6.90'}
{'Week': '2', 'Opponent': 'NYG', 'Rank': 29, 'Fantasy': '6.00'}
{'Week': '3', 'Opponent': 'NYJ', 'Rank': 8, 'Fantasy': '19.30'}
{'Week': '4', 'Opponent': 'ARI', 'Rank': 32, 'Fantasy': '6.70'}
{'Week': '5', 'Opponent': 'PHI', 'Rank': 7, 'Fantasy': '12.00'}
{'Week': '6', 'Opponent': 'DET', 'Rank': 26, 'Fantasy': '12.90'}
{'Week': '7', 'Opponent': 'TAM', 'Rank': 4, 'Fantasy': '3.60'}
{'Week': '9', 'Opponent': 'BUF', 'Rank': 5, 'Fantasy': '3.30'}
{'Week': '10', 'Opponent': 'NWE', 'Rank': 1, 'Fantasy': '5.70'}
{'Week': '11', 'Opponent': 'CAR', 'Rank': 10, 'Fantasy': '2.60'}
{'Week': '12', 'Opponent': 'MIA', 'Rank': 3, 'Fantasy': '0.70'}
{'Week': '13', 'Opponent': 'PHI', 'Rank': 7, 'Fantasy': '8.10'}
{'Week': '15', 'Opponent': 'NYG', 'Rank': 29, 'Fantasy': '3.60'}
{'Week': '1', 'Opponent': 'BAL', 'Ran

...

{'Week': '16', 'Opponent': 'GNB', 'Rank': 26, 'Fantasy': '8.90'}
{'Week': '1', 'Opponent': 'CAR', 'Rank': 19, 'Fantasy': '14.60'}
{'Week': '2', 'Opponent': 'NYG', 'Rank': 23, 'Fantasy': '14.70'}
{'Week': '3', 'Opponent': 'SEA', 'Rank': 11, 'Fantasy': '11.80'}
{'Week': '4', 'Opponent': 'DET', 'Rank': 16, 'Fantasy': '30.00'}
{'Week': '5', 'Opponent': 'HOU', 'Rank': 4, 'Fantasy': '8.40'}
{'Week': '6', 'Opponent': 'JAX', 'Rank': 5, 'Fantasy': '17.70'}
{'Week': '8', 'Opponent': 'TEN', 'Rank': 3, 'Fantasy': '11.20'}
{'Week': '9', 'Opponent': 'PHI', 'Rank': 12, 'Fantasy': '30.70'}
{'Week': '10', 'Opponent': 'ATL', 'Rank': 25, 'Fantasy': '26.10'}
{'Week': '13', 'Opponent': 'PHI', 'Rank': 12, 'Fantasy': '19.20'}
{'Week': '14', 'Opponent': 'IND', 'Rank': 10, 'Fantasy': '12.80'}
{'Week': '15', 'Opponent': 'TAM', 'Rank': 31, 'Fantasy': '10.90'}
{'Week': '1', 'Opponent': 'JAX', 'Rank': 5, 'Fantasy': '18.80'}
{'Week': '2', 'Opponent': 'DAL', 'Rank': 6, 'Fantasy': '10.80'}
{'Week': '3', 'Opponent': '

{'Week': '10', 'Opponent': 'ATL', 'Rank': 25, 'Fantasy': '32.90'}
{'Week': '11', 'Opponent': 'CIN', 'Rank': 30, 'Fantasy': '24.80'}
{'Week': '12', 'Opponent': 'HOU', 'Rank': 4, 'Fantasy': '13.20'}
{'Week': '13', 'Opponent': 'CAR', 'Rank': 19, 'Fantasy': '14.30'}
{'Week': '14', 'Opponent': 'DEN', 'Rank': 13, 'Fantasy': '10.00'}
{'Week': '15', 'Opponent': 'CIN', 'Rank': 30, 'Fantasy': '11.50'}
{'Week': '16', 'Opponent': 'BAL', 'Rank': 2, 'Fantasy': '2.00'}
{'Week': '1', 'Opponent': 'SEA', 'Rank': 11, 'Fantasy': '16.20'}
{'Week': '2', 'Opponent': 'OAK', 'Rank': 32, 'Fantasy': '11.10'}
{'Week': '3', 'Opponent': 'BAL', 'Rank': 2, 'Fantasy': '2.00'}
{'Week': '4', 'Opponent': 'KAN', 'Rank': 24, 'Fantasy': '13.90'}
{'Week': '5', 'Opponent': 'NYJ', 'Rank': 29, 'Fantasy': '8.10'}
{'Week': '6', 'Opponent': 'LAR', 'Rank': 20, 'Fantasy': '6.60'}
{'Week': '7', 'Opponent': 'ARI', 'Rank': 26, 'Fantasy': '15.60'}
{'Week': '8', 'Opponent': 'KAN', 'Rank': 24, 'Fantasy': '17.20'}
{'Week': '9', 'Opponent':

{'Week': '11', 'Opponent': 'NWE', 'Rank': 7, 'Fantasy': '6.00'}
{'Week': '12', 'Opponent': 'TEN', 'Rank': 3, 'Fantasy': '10.70'}
{'Week': '13', 'Opponent': 'BUF', 'Rank': 18, 'Fantasy': '0.70'}
{'Week': '1', 'Opponent': 'SFO', 'Rank': 28, 'Fantasy': '4.20'}
{'Week': '2', 'Opponent': 'GNB', 'Rank': 22, 'Fantasy': '1.90'}
{'Week': '3', 'Opponent': 'BUF', 'Rank': 18, 'Fantasy': '3.10'}
{'Week': '4', 'Opponent': 'LAR', 'Rank': 20, 'Fantasy': '3.80'}
{'Week': '5', 'Opponent': 'PHI', 'Rank': 12, 'Fantasy': '5.60'}
{'Week': '6', 'Opponent': 'ARI', 'Rank': 26, 'Fantasy': '21.80'}
{'Week': '7', 'Opponent': 'NYJ', 'Rank': 29, 'Fantasy': '20.30'}
{'Week': '9', 'Opponent': 'DET', 'Rank': 16, 'Fantasy': '10.70'}
{'Week': '10', 'Opponent': 'CHI', 'Rank': 1, 'Fantasy': '0.50'}
{'Week': '11', 'Opponent': 'GNB', 'Rank': 22, 'Fantasy': '3.30'}
{'Week': '12', 'Opponent': 'NWE', 'Rank': 7, 'Fantasy': '1.40'}
{'Week': '13', 'Opponent': 'SEA', 'Rank': 11, 'Fantasy': '0.40'}
{'Week': '14', 'Opponent': 'MIA',

{'Week': '10', 'Opponent': 'PHI', 'Rank': 15, 'Fantasy': '4.40'}
{'Week': '11', 'Opponent': 'DAL', 'Rank': 11, 'Fantasy': '8.50'}
{'Week': '12', 'Opponent': 'HOU', 'Rank': 19, 'Fantasy': '4.50'}
{'Week': '13', 'Opponent': 'KAN', 'Rank': 7, 'Fantasy': '0.90'}
{'Week': '14', 'Opponent': 'CIN', 'Rank': 25, 'Fantasy': '10.30'}
{'Week': '15', 'Opponent': 'BUF', 'Rank': 2, 'Fantasy': '10.10'}
{'Week': '16', 'Opponent': 'MIA', 'Rank': 32, 'Fantasy': '13.40'}
{'Week': '2', 'Opponent': 'JAX', 'Rank': 21, 'Fantasy': '9.00'}
{'Week': '3', 'Opponent': 'LAC', 'Rank': 14, 'Fantasy': '7.90'}
{'Week': '4', 'Opponent': 'CAR', 'Rank': 31, 'Fantasy': '6.40'}
{'Week': '5', 'Opponent': 'ATL', 'Rank': 23, 'Fantasy': '12.00'}
{'Week': '6', 'Opponent': 'KAN', 'Rank': 7, 'Fantasy': '17.00'}
{'Week': '7', 'Opponent': 'IND', 'Rank': 18, 'Fantasy': '3.50'}
{'Week': '8', 'Opponent': 'OAK', 'Rank': 24, 'Fantasy': '8.30'}
{'Week': '9', 'Opponent': 'JAX', 'Rank': 21, 'Fantasy': '14.00'}
{'Week': '10', 'Opponent': 'BA

{'Week': '1', 'Opponent': 'HOU', 'Rank': 19, 'Fantasy': '10.70'}
{'Week': '2', 'Opponent': 'LAR', 'Rank': 17, 'Fantasy': '2.30'}
{'Week': '3', 'Opponent': 'SEA', 'Rank': 22, 'Fantasy': '0.20'}
{'Week': '4', 'Opponent': 'DAL', 'Rank': 11, 'Fantasy': '2.00'}
{'Week': '5', 'Opponent': 'TAM', 'Rank': 29, 'Fantasy': '3.40'}
{'Week': '6', 'Opponent': 'JAX', 'Rank': 21, 'Fantasy': '7.90'}
{'Week': '7', 'Opponent': 'CHI', 'Rank': 4, 'Fantasy': '27.00'}
{'Week': '8', 'Opponent': 'ARI', 'Rank': 28, 'Fantasy': '27.70'}
{'Week': '9', 'Opponent': 'ATL', 'Rank': 23, 'Fantasy': '1.90'}
{'Week': '10', 'Opponent': 'TAM', 'Rank': 29, 'Fantasy': '3.20'}
{'Week': '11', 'Opponent': 'CAR', 'Rank': 31, 'Fantasy': '12.40'}
{'Week': '12', 'Opponent': 'ATL', 'Rank': 23, 'Fantasy': '0.20'}
{'Week': '13', 'Opponent': 'SFO', 'Rank': 8, 'Fantasy': '9.40'}
{'Week': '14', 'Opponent': 'IND', 'Rank': 18, 'Fantasy': '4.90'}
{'Week': '15', 'Opponent': 'TEN', 'Rank': 12, 'Fantasy': '2.50'}
{'Week': '16', 'Opponent': 'CAR'

In [62]:
gamedata.to_csv('WeekRankFantasy.csv')

In [ ]:
url_text = requests.get(f'https://www.pro-football-reference.com/players/C/GordMe00/fantasy/2017/').text
soup = BeautifulSoup(url_text)
name = soup.find_all('span')[7].text
print(name)

In [51]:
url_text = requests.get(f'https://www.pro-football-reference.com/players/J/JohnLa00/fantasy/2008/').text
soup = BeautifulSoup(url_text)
points = soup.find_all('td',attrs={"class":"right ","data-stat":"fantasy_points"})[15].text
print(points)

IndexError: list index out of range

In [61]:
gamedata

,Week,Opponent,Rank,Fantasy
0,1,DET,31.0,23.10
1,2,IND,7.0,25.40
2,3,NYJ,14.0,23.40
3,4,KAN,28.0,16.60
4,5,NWE,17.0,8.70
5,6,DEN,15.0,19.90
6,7,BUF,27.0,11.90
7,8,GNB,12.0,7.00
8,9,NYJ,14.0,6.90
9,10,BAL,19.0,25.00


In [14]:
finaldata = pd.read_csv(r'C:\Users\Parker Hill\Desktop\ECE 196\NFL Running Back\WeekRankFantasy.csv')
data = pd.DataFrame(finaldata)
data = data.drop(columns = ['Unnamed: 0','Opponent'])
print(data)
data.to_csv('FinalData.csv')

      Week  Rank  Fantasy
0        1    31     23.1
1        2     7     25.4
2        3    14     23.4
3        4    28     16.6
4        5    17      8.7
5        6    15     19.9
6        7    27     11.9
7        8    12      7.0
8        9    14      6.9
9       10    19     25.0
10      11    22     27.1
11      12    27     35.5
12      13    25     31.6
13      14     6     12.0
14      15    30      8.3
15      16    17     31.2
16       1    32     21.9
17       2    20      9.0
18       3    29     23.6
19       4    17     35.7
20       5    15      7.7
21       6    28     21.6
22       7     6     30.1
23       8    14     13.3
24       9    24     20.4
25      10    18     14.5
26      11     4      5.9
27      12    15     45.1
28      13     6     13.9
29      14    27     18.3
...    ...   ...      ...
5623    14     5      2.4
5624    15    24     16.7
5625    16     7     18.2
5626     1    16      9.8
5627     2    30     11.7
5628     6    32      2.6
5629     7  

In [44]:
#Making data into useable numpy arrays to be passed into the final dataset creation filter
data = pd.read_csv(r'C:\Users\Parker Hill\Desktop\ECE 196\NFL Running Back\FinalData.csv') #Change to directory that your data is in
data = pd.DataFrame(data)
length = len(data['Week'])
previousweeknum = -1
array = np.empty([1,3])
for i in range(0,length):
    weeknum = data['Week'][i]
    if weeknum < previousweeknum:
        array = np.delete(array,(0),axis=0) #I dunno how to make a truly empty array, so need to delete the first row each time
        #This line you would call your make data points function on with the created array, I used a print statement for verification
        print(array)
        array = np.empty([1,3])
    rank = data['Rank'][i]
    fantasy = data['Fantasy'][i]
    newrow = np.array([weeknum, rank, fantasy])
    array = np.vstack([array, newrow])
    previousweeknum = weeknum    

[[ 1.  31.  23.1]
 [ 2.   7.  25.4]
 [ 3.  14.  23.4]
 [ 4.  28.  16.6]
 [ 5.  17.   8.7]
 [ 6.  15.  19.9]
 [ 7.  27.  11.9]
 [ 8.  12.   7. ]
 [ 9.  14.   6.9]
 [10.  19.  25. ]
 [11.  22.  27.1]
 [12.  27.  35.5]
 [13.  25.  31.6]
 [14.   6.  12. ]
 [15.  30.   8.3]
 [16.  17.  31.2]]
[[ 1.  32.  21.9]
 [ 2.  20.   9. ]
 [ 3.  29.  23.6]
 [ 4.  17.  35.7]
 [ 5.  15.   7.7]
 [ 6.  28.  21.6]
 [ 7.   6.  30.1]
 [ 8.  14.  13.3]
 [ 9.  24.  20.4]
 [10.  18.  14.5]
 [11.   4.   5.9]
 [12.  15.  45.1]
 [13.   6.  13.9]
 [14.  27.  18.3]
 [15.  28.  15.7]
 [16.  23.   8.5]]
[[ 1.   2.  20. ]
 [ 2.  13.   8.6]
 [ 3.  10.  15.4]
 [ 4.   6.   9.9]
 [ 6.   9.  26. ]
 [ 7.  32.   9.2]
 [ 8.   7.  11.1]
 [ 9.  20.   8.6]
 [10.  16.  21.3]
 [11.  19.   7.2]
 [12.   3.  14.3]
 [13.   7.  17.9]
 [14.  17.  11.7]
 [15.   9.  20.8]
 [16.  20.  18. ]]
[[ 1.  14.  33.8]
 [ 2.  30.   3. ]
 [ 3.  15.  10.8]
 [ 4.  25.  14.9]
 [ 5.   6.  16.5]
 [ 6.  20.  27. ]
 [ 7.   4.  13.7]
 [ 8.  31.  17.6]
 [ 9.  

[[ 1.   6.  16.4]
 [ 3.   5.   7. ]
 [ 5.  23.  13.3]
 [ 7.  26.   2.9]
 [ 8.   4.   6.7]
 [ 9.   1.  16.7]
 [10.  16.  21.1]
 [11.  24.   3.1]
 [12.  23.  10.2]
 [13.  26.  21.5]
 [14.   4.   3.9]
 [15.  10.  13.5]
 [16.   2.  10.5]]
[[ 1.   3.  13.2]
 [ 3.  30.  17.1]
 [ 6.   5.  17.8]
 [ 7.  29.  25.3]
 [ 8.  10.  13. ]
 [ 9.  14.   5.5]
 [10.   2.   2.8]
 [11.  19.  14.3]
 [12.   6.  14. ]
 [13.  32.  15. ]
 [14.  30.  12.3]
 [15.  14.   6.9]
 [16.   3.  11. ]]
[[ 1.   2.   9.4]
 [ 2.   5.   9.7]
 [ 3.   6.  15.3]
 [ 4.  14.  14.2]
 [ 5.   3.   6.5]
 [ 6.  30.   8. ]
 [ 7.   2.  12. ]
 [ 8.  25.   1.5]
 [11.   4.  19.1]
 [12.   7.   8.7]
 [13.   6.  18.5]
 [14.   5.   5.9]
 [15.  31.  17.5]
 [16.   1.   0.3]]
[[ 1.  30.   3.2]
 [ 2.  29.  15.8]
 [ 3.  24.  11.9]
 [10.  32.   3.9]
 [11.  25.  20.1]
 [12.   9.  15.2]
 [13.  29.   6.2]
 [14.  32.  13.3]
 [15.   7.  26.2]
 [16.  21.  14.6]]
[[ 1.  14.   5.3]
 [ 2.  20.   9.7]
 [ 3.  26.  21.1]
 [ 4.   4.   4.1]
 [ 5.  24.   1.7]
 [ 6. 

 [16.   7.  19.4]]
[[ 1.   5.   8.7]
 [ 2.  26.   2.2]
 [ 4.  16.  19.1]
 [ 5.  12.  16.1]
 [ 6.  23.   4.7]
 [ 7.  28.   1. ]
 [ 8.  16.   6.6]
 [ 9.   6.  16. ]
 [10.   8.  11.8]
 [11.   4.  10.7]
 [12.  18.  13.8]
 [13.  19.  22.5]
 [14.  28.   2.8]
 [15.  21.  27. ]
 [16.   4.   6.7]]
[[ 1.  26.  15.4]
 [ 2.   6.   2.3]
 [ 3.   7.   4.9]
 [ 4.   2.   3.4]
 [ 5.   9.   4.5]
 [ 6.  21.  34.1]
 [ 7.  10.  14.8]
 [ 8.   7.  10.6]
 [ 9.   1.   6.3]
 [10.  15.   6.4]
 [11.  10.   6. ]
 [12.  17.  10.4]
 [13.  30.  13.7]
 [14.  32.   6.9]
 [15.  16.  20. ]
 [16.  26.   7.7]]
[[ 1.  29.   4.4]
 [ 2.   8.  26.5]
 [ 3.  26.  11.1]
 [ 4.  12.  16.4]
 [ 5.  13.  18.8]
 [ 6.   8.   5.4]
 [ 7.  21.   1.9]
 [ 8.  16.  19.4]
 [ 9.  12.  13.6]
 [10.  24.  14.4]
 [11.   6.  17.2]
 [12.  28.  15.5]
 [13.  25.  17.1]
 [15.   4.   6.8]]
[[ 1.  11.  17.3]
 [ 2.  27.  16.5]
 [ 3.  14.   5.2]
 [ 4.  20.  13.7]
 [ 5.  26.  16.6]
 [ 6.  13.   5.6]
 [ 7.   1.  18.2]
 [ 8.  17.   6.6]
 [ 9.  15.  17.5]
 [10. 

 [13.   3.   0.3]]
[[ 2.  15.  12.7]
 [ 3.  27.  15.6]
 [ 4.   3.   7.4]
 [ 5.  19.   8.7]
 [ 6.  11.   2.9]
 [ 8.  25.   3.7]
 [ 9.  23.   5.7]
 [10.  13.   7.7]
 [11.   9.   2.6]
 [12.  14.   2.3]
 [13.  12.   2.1]
 [14.  27.   1.5]
 [15.  20.   4.8]]
[[ 1.  13.   1.3]
 [ 2.  21.   2. ]
 [ 3.   4.   3.3]
 [ 5.   3.  13.2]
 [ 6.  23.  13. ]
 [ 7.  14.   4.8]
 [ 8.  12.   7.9]
 [ 9.  25.  10.8]
 [10.  15.  15. ]
 [12.  20.   9.1]
 [13.  13.   4.4]
 [14.   4.   7.6]
 [15.  19.   1. ]]
[[ 1.  19.   4.1]
 [ 2.  27.   8.6]
 [ 3.   8.   8.6]
 [ 4.   1.  16.9]
 [ 5.  11.   3.4]
 [ 7.   7.   3.8]
 [ 8.  26.   9.7]
 [ 9.  31.  11.5]
 [10.   9.   9.5]
 [11.   1.   4.3]
 [14.   9.   2.1]
 [15.  16.   2.6]
 [16.  26.   8.9]]
[[ 1.  19.  14.6]
 [ 2.  23.  14.7]
 [ 3.  11.  11.8]
 [ 4.  16.  30. ]
 [ 5.   4.   8.4]
 [ 6.   5.  17.7]
 [ 8.   3.  11.2]
 [ 9.  12.  30.7]
 [10.  25.  26.1]
 [13.  12.  19.2]
 [14.  10.  12.8]
 [15.  31.  10.9]]
[[ 1.   5.  18.8]
 [ 2.   6.  10.8]
 [ 3.   4.  17.7]
 [ 5.

In [26]:
array = np.empty([1,3])
newrow = np.array([1,2,3])
array = np.vstack([array,newrow])
print(array)

[[0.e+000 5.e-324 0.e+000]
 [1.e+000 2.e+000 3.e+000]]
